<a href="https://colab.research.google.com/github/dcsan/dalle-playground/blob/main/backend/dalle_playground_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/285/woman-artist_1f469-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [1]:
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!pip install flask-cloudflared

Cloning into 'dalle-playground'...
remote: Enumerating objects: 651, done.
remote: Counting objects: 100% (651/651), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 651 (delta 394), reused 562 (delta 359), pack-reused 0
Receiving objects: 100% (651/651), 2.47 MiB | 11.24 MiB/s, done.
Resolving deltas: 100% (394/394), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-1mwttmga
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-1mwttmga
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/patil-suraj/vqgan-jax.git to /tmp/pip-req-build-0sgtjd36
  Running command git clone -q https://github.com/patil-suraj/vqgan-jax.git /tmp/pip-req-build-0sgtjd36
  Cloning https://github.com/borisdayma/d

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [2]:
#@title Select a model - remember to execute cell after selecting!

dalle_model = 'Mega_full' #@param ["Mini", "Mega", "Mega_full"]

# Run the backend web server

In [ ]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port,))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  print(f"Selected DALL-E Model - [{dalle_model}]")
  !python dalle-playground/backend/app.py --port {app_port} --model_version {dalle_model} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    t1.join()

Selected DALL-E Model - [Mega_full]
--> Starting DALL-E Server. This might take up to two minutes.
Your url is: https://nuclear-enemies-fleece-separately.trycloudflare.com
tcmalloc: large alloc 10350092288 bytes == 0x7ff5ce1f6000 @  0x7ff98b5141e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x4bc98a 0x532b86 0x594a96 0x515600 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7ff98b111c87 0x5b621a
Downloading: 100% 34.2M/34.2M [00:00<00:00, 80.3MB/s]
/usr/local/lib/python3.7/dist-packages/jax/_src/ops/scatter.py:90: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float16 to dtype=float32. In future JAX releases this will result in an error.
  FutureWarning)
--> DALL-E Server is up and running!
--> Model selected - DALL-E ModelSize.MEGA_FULL
 * Serving Flask app 'app' (lazy loading)
 * Environment: production
   Use a produ

### Now, take the url you got from cloudflare (should look like `your url is: https://xxxxxx.trycloudflare.com`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.trycloudflare.com and run it in the browser.

### Let the fun begin ✨